<div class="alert alert-block alert-info">
<h3>Student Information</h3> Please provide information about yourself. We will NOT grade this submissing w/o all the information<br>
<b>Name</b>:Yinfeng Cong<br>
<b>NetID</b>:yc957<br>
<b>Recitation (01,02,03)</b>:02<br>
<b>Notes to Grader</b> (optional):<br>
<br><br>
<b>IMPORTANT</b>
Your work will not be graded withour your initials below<br>
I certify that this lab represents my own work and I have read the RU academic intergrity policies at<br>
<a href="https://www.cs.rutgers.edu/academic-integrity/introduction">https://www.cs.rutgers.edu/academic-integrity/introduction </a><br>
<b>Initials</b>: YC     


<h3>Grader Notes</h3>
<b>Your Grade<b>:<br>
<b>Grader Initials</b>:<br>
<b>Grader Comments</b> (optional):<br>
</div>

## CS 439 - Introduction to Data Science
### Spring 2022

# Lab 3: Exploratary Data Analysis

### Due Date: Thursday March 03, 2022 by 11:59 PM ###

### Instructions
This lab is presented as a notebook. Please execute the cells that are already completed and your task is to fill in the code
between ### BEGIN SOLUTION ### and ### END SOLUTION ###. 

#### Important: Please do not add any new cells or change the order of cells. If you have questions, please contact the courseS staff.

In this lab, you will be working with a dataset from NYPD containing data on calls to the New York Police Department. Information about the datasets can be found at https://opendata.cityofnewyork.us/

## Setup

Note that after activating matplotlib to display figures inline via the IPython magic `%matplotlib inline`, we configure a custom default figure size. Virtually every default aspect of matplotlib [can be customized](https://matplotlib.org/users/customizing.html).

In [ ]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)

# Part 1:Getting Data

We will work with the NYPD Historic complaint data set. Our first task is to estimate the size of this download by looking at the number of rows, columns and using an estimated size for a column (use a reasonable value). The site metadata is available from the page
https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i

In [ ]:
### BEGIN SOLUTION
estimated size of the download (based on metadata information from the site) =  2.3 GB     
# please explain how you reached the answer
#There are 21 columns with texts, data type is object, assuming 8 bytes for each data.
#There are 10 columns with numbers, assuming 4 bytes for each data.
#There are 3 columns with dates, assuming 8 bytes for each data.
#There is 1 column with location, assuming 64 bytes for each data.
#Total: 7370000*(10*4+21*8+3*8+1*64)/1000000000 = 2.182
### END SOLUTION

### 1.1 Download the data
This file is large (use the estimate you did above). If it takes too long to download, you may want to interrupt and download the file using a browser and URL https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i

In [ ]:
import utils
data_dir = 'data'
data_url = 'https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i'

file_name = 'NYPD_Complaint_Data_Historic.csv'

# To retrieve the dataset, we will use the `utils.fetch_and_cache` utility from utils library. 
dest_path = utils.fetch_and_cache(data_url=data_url, file=file_name,data_dir=data_dir)
print(f'Located at {dest_path}')

### 1.2 Inspect the size of the file
It is helpful to get an idea of the size of the file. This can be done using functions in the utils library.

In [ ]:
# look at the size of the file w/o opening it using OS (https://docs.python.org/3/library/os.html). You can perform
# variety of operating system related functions from this package.
### BEGIN SOLUTION
import os
size = os.stat('data/NYPD_Complaint_Data_Historic.csv')
size.st_size

#### END SOLUTION

### 1.3 Split the large file
This data file NYPD_Complaint_Data_Historic.csv is too big to load into a single DataFrame. Let us split the large file into smaller files.  Let us find out the number of lines in the NYPD_Complaint_Data_Historic.csv file using utils.

In [ ]:
# Using utils library, find the number of lines in the file

### BEGIN SOLUTION
from utils import *
linecount = line_count('data/NYPD_Complaint_Data_Historic.csv')
linecount

### END SOLUTION

In [ ]:
# Split the file into 10 smaller files. Estimate the number of lines in each file using the count above
# files should be created in the data folder and named NYPD_Complaint_Data_Historic_1.csv, 
# NYPD_Complaint_Data_Historic_2.csv, ... NYPD_Complaint_Data_Historic_10.csv etc
# It is possible that few lines from the original file may not be saved due to rounding errors.

### BEGIN SOLUTION
import pandas as pd
import itertools

# obtain line number range for each sub-dataset
sub_line_num = int(np.ceil(float(linecount) / 10)) # this way can bypass the rounding error and save everything

# use iterator to write line by line, instead of reading large file to memory
filename = 'data/NYPD_Complaint_Data_Historic.csv'
for i in range(10):
    with open(filename, "r") as f:
        fw = open('data/NYPD_Complaint_Data_Historic_%d.csv' % (i+1), "w")
       
        sliced_f = itertools.islice(f, i*sub_line_num, min((i+1)*sub_line_num, linecount))  
        # iterate through the sliced lines and write each line
        for line in sliced_f:
            fw.write(line)


### END SOLUTION

### 1.4 Check the files in the data folder
Now, we'll use a method of the `Pathlib.Path` class called `glob` to list all files in the `data` directory. You will find useful information in pathlib [docs](https://docs.python.org/3/library/pathlib.html).

Below, we use pathlib's `glob` method to store the list of all files' names from the `data_dir` directory in the variable `file_names`. These names should be strings that contain only the file name (e.g. `dummy.txt` not `data/dummy.txt`). The asterisk (*) character is used with the `glob` method to match any string.

In [ ]:
from pathlib import Path
data_dir_path = Path('data') # creates a Path object that points to the data directory
file_names = [x.name for x in data_dir_path.glob('*') if x.is_file()]
file_names

### 1.5 Pre-processing of data
It is good to pre-process the data to see if the file can be opened in a Jupyter notebook. We need to avoid large files that can crash notebooks. Typically, files of size around 200 MB is ok to open into a DataFrame. In the following activities we will inspect the file w/o opening it as a DataFrame. Using utils.head 

In [ ]:
# Use the util.head() to read up to 5 lines from the original file (w/o opening it)
from utils import head
head('data/NYPD_Complaint_Data_Historic.csv')

### 1.6 Inspecting and describing data columns
There should be 35 columns in each record. Using header information and data types, describe the type of data in each
column. If you are unable to determine, just state so.
##### BEGIN SOLUTION
* CMPLNT_NUM : number-integer
* CMPLNT_FR_DT: Floating Timestamp-datetime
* CMPLNT_FR_TM:Text-Object
* CMPLNT_TO_DT:Floating Timestamp-datetime
* CMPLNT_TO_TM:Text-Object
* ADDR_PCT_CD:number-integer
* RPT_DT:Timestamp-datetime
* KY_CD: number-integer
* OFNS_DESC:Text-Object
* PD_CD: number-integer
* PD_DESC: Text-Object
* CRM_ATPT_CPTD_CD: Text-Object
* LAW_CAT_CD: Text-Object
* BORO_NM: Text-Object
* LOC_OF_OCCUR_DESC: Text-Object
* PREM_TYP_DESC: Text-Object
* JURIS_DESC: Text-Object
* JURISDICTION_CODE: number-integer
* PARKS_NM: Text-Object
* HADEVELOPT: Text-Object
* HOUSING_PSA: Text-Object
* X_COORD_CD: number-float
* Y_COORD_CD: number-float
* SUSP_AGE_GROUP: Text-Object
* SUSP_RACE: Text-Object
* SUSP_SEX: Text-Object
* TRANSIT_DISTRICT: number-integer
* Latitude: number-float
* Longitude: number-float
* Lat_Lon: location
* PATROL_BORO: Text-Object
* STATION_NAME: Text-Object
* VIC_AGE_GROUP: Text-Object
* VIC_RACE:Text-Object
* VIC_SEX:Text-Object
##### END SOLUTION

### Part 2 - Exploratory Data Analysis
Exploratory data analysis (EDA) is the process of examining a subest of a large data set to see what we can know about the data. First we will explore one file NYPD_Complaint_Data_Historic_1.csv to see what we can find out.

### 2.1 Time to load dta into a DataFrame
Load the first CSV file, NYPD_Complaint_Data_Historic_1.csv into a `pandas.DataFrame` object. Also do a time analysis to see how long it took to load the data into a DataFrame. Time should be printed in seconds. The time libraries https://docs.python.org/3/library/time.html can help.

In [ ]:
!pip install memory_profiler

In [ ]:
%load_ext memory_profiler

In [ ]:
### BEGIN SOLUTION
import pandas as pd
import time

time_start=time.time()
calls=pd.read_csv('data/NYPD_Complaint_Data_Historic_1.csv')
time_end=time.time()
print('time cost',time_end-time_start,'s')
%memit pd.read_csv('data/NYPD_Complaint_Data_Historic_1.csv')
calls.head()


### END SOLUTION

### 2.2 Description of Fields
Let's also check some basic information about these files using the `DataFrame.describe` and `DataFrame.info` methods. Describe columns that can be removed based on the information.

In [ ]:
### BEGIN SOLUTION
calls.info()
calls.describe()
calls.columns[calls.isnull().mean() > 0.5].values
# What columns can be removed from the DataFrame? A reasonable rule of thumb is that if a column is missing more than 
# 50% of the data then it should be removed

### END SOLUTION

### 2.2 Finding Uniques

Notice that the functions above reveal type information for the columns, as well as some basic statistics about the numerical columns found in the DataFrame. However, we still need more information about what each column represents. Let's explore the data further.

find the number of unique values in each DataFrame column and answer the questions below.

In [ ]:
### BEGIN SOLUTION
calls.nunique()

# Questions
# 1. How many distinct locations where the complaints have come from? - 77 (ADDR_PCT_CD)
# 2. How many age groups are represented in the data set? - 25 (SUSP_AGE_GROUP)
# 3. How many boroughs are included in the data set? - 5 (BORO_NM)
# 4. How many offense types are listed in this data set? - 69 (KY_CD)

### END SOLUTION

### 2.3 Offense by Boro
Using GroupBy operation, create a DataFrame that groups offenses by Boro. call the DataFrame calls_by_Boro_and_offense

In [ ]:
### BEGIN SOLUTION
calls_by_Boro_and_offense = calls.groupby('BORO_NM').groups
calls_by_Boro_and_offense
### END SOLUTION

### 2.4 Offenses in Bronx

In the cell below, find a list of strings corresponding to the possible values for `OFNS_DESC` when `BORO` is "BRONX". Create an expression that automatically extracts the names of the offenses.

In [ ]:
### BEGIN SOLUTION
calls_by_Boro_and_offense = calls.groupby('BORO_NM')['OFNS_DESC'].apply(pd.DataFrame)
print(calls_by_Boro_and_offense['BRONX'].dropna().unique())

calls_by_Boro_and_offense = calls.groupby('BORO_NM')
print(len(calls_by_Boro_and_offense.get_group('BRONX')))

### END SOLUTION

# How many offenses were committed in Bronx during the analysis period?
#160808

### 2.5 Most Common Crimes in NYC

What are the five crime types of OFNS_DESC that have the most crime events in Bronx? You may need to use `value_counts` to find the answer. Save your results as a list of strings.

**Hint:** *The `keys` method of the Series class might be useful.*

In [ ]:
### BEGIN SOLUTION
calls_by_Boro_and_offense = calls.groupby('BORO_NM')['OFNS_DESC'].apply(pd.DataFrame)
calls_by_Boro_and_offense['BRONX'].value_counts().head(5).index.unique()

### END SOLUTION

### 2.6 Probability of a Crime in Bronx
What is the probability that a the crime "Arson" can happen in Bronx?

In [ ]:
### BEGIN SOLUTION
p = calls_by_Boro_and_offense['BRONX'].value_counts()['ARSON']/calls_by_Boro_and_offense['BRONX'].count().sum()
p
### END SOLUTION

---
# Part 3: Visualizing the Data

## Pandas vs. Seaborn Plotting

Pandas offers basic functionality for plotting. For example, the `DataFrame` and `Series` classes both have a `plot` method. However, the basic plots generated by pandas are not particularly pretty. While it's possible to manually use matplotlib commands to make pandas plots look better, we'll instead use a high level plotting library called Seaborn that will take care of most of this for us.

As you learn to do data visualization, you may find the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html) and [Seaborn documentation](https://seaborn.pydata.org/api.html) helpful!

We will continue use EDA for examining a subest of a large data set to see what we can know about the data. Continue to explore one file NYPD_Complaint_Data_Historic_1.csv to see what we can find out.

### 3.1 Plotting a Series
Using the built-in plotting functionality of pandas, such as `plot` method of the `Series` class to generate a `barh` plot type,  display the value counts for `OFNS_DESC` visually as a barh chart.

In [ ]:
### BEGIN SOLUTION
calls_by_Boro_and_offense['BRONX'].value_counts().plot.barh()
plt.xlabel('Number Of Calls')
plt.ylabel('Crime Category')
plt.title('Number Of Calls For Each Offense')
plt.show()


### END SOLUTION

### 3.2 Getting a Better Plot
The plot above can be messy as it plots all offenses. Plot only the offenses that has more than 10000 calls

In [ ]:
### BEGIN SOLUTION
calls_by_Boro_and_offense['BRONX'].value_counts()[calls_by_Boro_and_offense['BRONX'].value_counts()>10000].plot.barh()
plt.xlabel('Number Of Calls')
plt.ylabel('Crime Category')
plt.title('Number of Calls By Crime Type')
plt.show()

### END SOLUTION

By contrast, the Seaborn library provides a specific function `countplot` built for plotting counts. It operates directly on the DataFrame itself i.e. there's no need to call `value_counts()` at all. This higher level approach makes it easier to work with. Use the y-label ("Crime Category"), x-label("Number of Calls") and title_of_plot("Number of Calls By Crime Type"

In [ ]:
### BEGIN SOLUTION
plot = sns.countplot(y = 'OFNS_DESC', data=calls)

plt.xlabel("Number of Calls")
plt.ylabel("Crime Category")
plt.title("Number of Calls By Crime Type")
plt.show()

### END SOLUTION

You may note that the ordering might be different for the seaborn plot (as compared to pandas plot). If we want the same ordering that we had in the pandas plot, we can use the order parameter of the `countplot` method. It takes a list of strings corresponding to the axis to be ordered. By passing the index of the `value_counts`, you can get the order you want.

In [ ]:
### BEGIN SOLUTION
plot = sns.countplot(y = 'OFNS_DESC',order = calls['OFNS_DESC'].value_counts(ascending = True).index, data=calls)

plt.xlabel("Number of Calls")
plt.ylabel("Crime Category")
plt.title("Number of Calls By Crime Type")
plt.show()

### END SOLUTION

Now we have a pretty bar plot with the bars ordered by size. Though `seaborn` appears to provide a superior plot from a aesthetic point of view, the `pandas` plotting library is also good to understand. You'll get practice using both libraries in the following questions.

## An Additional Note on Plotting in Jupyter Notebooks

You may have noticed that many of our code cells involving plotting end with a semicolon (;). This prevents any extra output from the last line of the cell that we may not want to see. Try adding this to your own code in the following questions!

### 3.3 making more plots

Now it is your turn to make some more plots using `pandas` and `seaborn`. Let's start by looking at the distribution of calls over days of the week.

The CMPLNT_FR_DT field contains the date of the event. We would like to add a new column to the DataFrame that includes Day of the week (DAY_OF_WEEK) that indicates the day of the week. This can help us analyze the crimes on a specific day of the week. For example, we can answer questions such as "what day of the week that a LARSON is likely to happen in NYC?"


Add a new column `DAY_OF_WEEK` into the `calls` dataframe that has the day string (eg. 'Sunday') for the corresponding value in CMPLNT_FR_DT. For example, if the first 3 values of `CMPLNT_FR_DT` are `['01/27/2006, '01/28/2006, '01/29/2006]`, then the first 3 values of the `DAY_OF_WEEK` column should be `["Friday", "Saturday", "Sunday"]`.

**Hint:** *Try using the [Series.map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function on `calls["OFNS_DESC"]`.  Can you assign this to the new column `calls["DAY_OF_WEEK"]`?*

In [ ]:
days = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
### BEGIN SOLUTION
date = pd.to_datetime(calls["CMPLNT_FR_DT"], errors = 'coerce')
week = date.dt.dayofweek
DAY_OF_WEEK = week.map({0.0:'Monday',1.0:'Tuesday',2.0:'Wednesday',3.0:'Thursday',4.0:'Friday',5.0:'Saturday',6.0:'Sunday'})
DAY_OF_WEEK.tolist()
calls['DAY_OF_WEEK'] = DAY_OF_WEEK.tolist()
calls

### END SOLUTION

### 3.4 Seaborn plots

Create a `seaborn` plot that shows the number of calls for each day of the week. You may want to use of the `rotation` argument in `ax.set_xticklabels`, which rotates the labels by 90 degrees.

In [ ]:
### BEGIN SOLUTION
plot = sns.countplot(x ='DAY_OF_WEEK', data=calls,order = ['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday'])
xlabels = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]

plot.set_xticklabels(labels = xlabels,rotation=90)
plt.ylabel('Number Of Calls')
plt.title('Number of Calls For Each Day of the Week')
plt.show()

### END SOLUTION

Now, let's make the same plot using `pandas`. Construct a vertical bar plot with the count of the number of calls (entries in the table) for each day of the week **ordered by the day of the week** (eg. `Sunday`, `Monday`, ...). Do not use `sns` for this plot. Be sure that your axes are labeled and that your plot is titled.

**Hint:** *Given a series `s`, and an array `coolIndex` that has the same entries as in `s.index`, `s[coolIndex]` will return a copy of the series in the same order as `coolIndex`.*

In [ ]:
### BEGIN SOLUTION
calls['DAY_OF_WEEK'].value_counts().reindex(["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]).plot.bar()
plt.xlabel('Day')
plt.ylabel('Number Of Calls')
plt.title('Number of Calls For Each Day of the Week')
plt.show()
### END SOLUTION

### 3.5 What Day of the Week is more calls?

Is it true that weekdays generally have slightly more calls than Saturday or Sunday? What can you say about the difference?

##### BEGIN SOLUTION

Yes, that can be say about most people prefer to saty at home for a rest in the weekend, and they need to go out for work during weekday. So there may have more probability in the weekday.

##### END SOLUTION

We can break down into some particular types of events to see their distribution. For example, let's make a bar plot for the OFNS_DESC "HARRASSMENT 2". Which day is the peak for "HARRASSMENT 2"?

This time, use `seaborn` to create a vertical bar plot of the number of total noise violations reported on each day of the week, again ordered by the days of the week starting with Sunday. Do not use `pandas` to plot.

**Hint:** *If you're stuck, use the code for the seaborn plot in above question as a starting point.*

In [ ]:
### BEGIN SOLUTION
plot = sns.countplot(x ='DAY_OF_WEEK', order = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"], data=calls[calls['OFNS_DESC'] == 'HARRASSMENT 2'])
plt.xlabel('DAY_OF_WEEK')
plot.set_xticklabels(labels = xlabels, rotation=90)
plt.ylabel ('Number of Calls')
plt.title('Number of Harrassment-2 Calls For Each Day of the Week' )
plt.show()
### END SOLUTION

### 3.6 Distribution

Do you see anything interesting about the distribution of HARRASSMENT 2 calls over a week? Type a short answer below.
##### BEGIN SOLUTION
HARRASSMENT 2 are less happen in the weekend, I guess they have weekend too, they don't have to work. So there may have more HARRASSMENT 2 in the weekday.
##### END SOLUTION

### 3.7 More Plots

Let's look at a similar distribution but for a crime we have much more calls data about. In the cell below, create the same plot as you did in previous questions, but now looking at instances of the OFNS_DESC "BURGLARY" (instead of "HARRASSMENT 2"). Use either `pandas` or `seaborn` plotting as you desire.

In [ ]:
### BEGIN SOLUTION
plot2 = sns.countplot(x ='DAY_OF_WEEK', order = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"], data=calls[calls['OFNS_DESC'] == 'BURGLARY'])
plt.xlabel ('DAY_OF_WEEK')
plot2.set_xticklabels(labels = xlabels, rotation=90)
plt.ylabel ('Number of Calls')
plt.title('Number of BURGLARY Calls For Each Day of the Week' )
plt.show()

### END SOLUTION



### 3.8 time of events

Now let's look at the CMPLNT_TO_TM column which indicates the time for events. Since it contains hour and minute information, let's extract the hour info and create a new column named `Hour` in the `calls` dataframe. You should save the hour as an `int`. Then plot the frequency of each hour in the table (i.e., `value_counts()`) sorted by the hour of the day (i.e., `sort_index()`).

You will want to look into how to use:

* [Series.str.slice](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.slice.html#pandas.Series.str.slice) to select the substring.
* [Series.astype](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.astype.html) to change the type.

**Hint:** *The `str` helper member of a series can be used to grab substrings.  For example, `calls["CMPLNT_TO_TM"].str.slice(3,5)` returns the minute of each hour of the `CMPLNT_TO_TM`.*

In [ ]:
calls["Hour"] = calls["CMPLNT_TO_TM"].str.slice(0,2).replace(np.NaN,0).astype('int')

In [ ]:
### BEGIN SOLUTION
calls['Hour'] = calls['CMPLNT_TO_TM'].str.slice(0,2).fillna(0).astype(int)
calls['Hour'][calls['Hour']!= -1].value_counts().sort_index().plot.bar(figsize = (12,12))
plt.xlabel ('Hour')
plt.ylabel ('Number of Calls')
plt.title('Number of Calls For per Hour' )
plt.show() 

### END SOLUTION

Create a pandas bar plot showing the number of BURGLARY crimes committed at each hour of the day. Use the labels
* ax.set_xlabel("Hour of the Day")
* ax.set_ylabel("Number of Calls")
* ax.set_title("Number of Calls Reporting Fraud For Each Day of the Week");

In [ ]:
### BEGIN SOLUTION

x = calls['Hour'][(calls['Hour']!= -1)&(calls['OFNS_DESC'] == 'BURGLARY')].value_counts().sort_index().plot.bar()
x.set_xlabel("Hour of the Day")
x.set_ylabel("Number of Calls")
x.set_title("Number of Calls Reporting Fraud For Each Day of the Week")
plt.show()

### END SOLUTION

### 3.9 More plots

In the cell below, create a seaborn plot of the same data. Again, make sure you provide axes labels and a title for your plot.

In [ ]:
### BEGIN SOLUTION
sns.countplot(x ='Hour', data=calls[(calls['Hour']!= -1)&(calls['OFNS_DESC'] == 'BURGLARY')])
plt.xlabel("Hour of the Day")
plt.ylabel("Number of Calls")
plt.title("Number of Calls Reporting Fraud For Each Day of the Week")
plt.show()
### END SOLUTION

### 3.10 Spike in burglary?

According to your plots, is there a spike in calls reporting BURGLARY at any particular time? If so, Do you trust that this spike is legitimate, or could there be an issue with our data? Explain your reasoning in 1-2 sentences below.

#### BEGIN SOLUTION
Yes, becasue the peak of hour of the day is 7 and 17, these were the time people week up or come back to their home after work who just find that something wrong with their house and reporting fraud.

#### END SOLUTION

In the cell below, we generate a boxplot which examines the hour of day of each crime broken down by the `OFNS_DESC` value.  To construct this plot we used the [DataFrame.boxplot](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.boxplot.html) documentation.

In [ ]:
calls.boxplot(column="Hour", by='OFNS_DESC', rot=90);

While the pandas boxplot is informative, we can use seaborn to create a more visually-appealing plot. Using seaborn, regenerate a better box plot. See either the textbook (https://www.textbook.ds100.org/ch/06/viz_quantitative.html) or the [seaborn boxplot documentation](https://seaborn.pydata.org/generated/seaborn.boxplot.html).

Looking at your plot, which crime type appears to have the largest interquartile range? Put your results into `answer` as a string.

In [ ]:
# Todo: Make a boxplot with seaborn
### BEGIN SOLUTION
ax = sns.boxplot(y = 'Hour', x = 'OFNS_DESC', data = calls)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
ax.set_title("Distributions of Calls Over Hours in a Day, Grouped by Crime");
answer = "Prostitution & related offenses ,Offense related to children, and Loitering/Gambling "
### END SOLUTION

### 3.11a Explore Suspect and Victim Age Relationships
In this activity we explore relations between SUSP_AGE_GROUP and VIC_AGE_GROUP
1. create a matrix of where rows are SUSP_AGE_GROUPS and Columns are VIC_AGE_GROUPs
2. Fill in the matrix with all probabilities (eg. "the probability that <18 susp_group target <18 victim_group). 

In [ ]:
### BEGIN SOLUTION
import numpy as np
matrix = np.empty([5,5],dtype = float)
M = calls.groupby('SUSP_AGE_GROUP')['VIC_AGE_GROUP'].apply(pd.DataFrame)
a = M['<18'].value_counts()['<18']
b = M['<18'].value_counts()['18-24']
c = M['<18'].value_counts()['25-44']
d = M['<18'].value_counts()['45-64']
e = M['<18'].value_counts()['65+']
f = M['18-24'].value_counts()['<18']
g = M['18-24'].value_counts()['18-24']
h = M['18-24'].value_counts()['25-44']
i = M['18-24'].value_counts()['45-64']
j = M['18-24'].value_counts()['65+']
k = M['25-44'].value_counts()['<18']
l = M['25-44'].value_counts()['18-24']
m = M['25-44'].value_counts()['25-44']
n = M['25-44'].value_counts()['45-64']
o = M['25-44'].value_counts()['65+']
p = M['45-64'].value_counts()['<18']
q = M['45-64'].value_counts()['18-24']
r = M['45-64'].value_counts()['25-44']
s = M['45-64'].value_counts()['45-64']
t = M['45-64'].value_counts()['65+']
u = M['65+'].value_counts()['<18']
v = M['65+'].value_counts()['18-24']
w = M['65+'].value_counts()['25-44']
x = M['65+'].value_counts()['45-64']
y = M['65+'].value_counts()['65+']
sum_of_age = a + b + c + d + e + f + g + h + i + j + k + l + m + n + o + p + q + r + s + t + u + v + w + x + y
matrix[0][0] = a/sum_of_age * 100
matrix[0][1] = b/sum_of_age * 100
matrix[0][2] = c/sum_of_age * 100
matrix[0][3] = d/sum_of_age * 100
matrix[0][4] = e/sum_of_age * 100
matrix[1][0] = f/sum_of_age * 100
matrix[1][1] = g/sum_of_age * 100
matrix[1][2] = h/sum_of_age * 100
matrix[1][3] = i/sum_of_age * 100
matrix[1][4] = j/sum_of_age * 100
matrix[2][0] = k/sum_of_age * 100
matrix[2][1] = l/sum_of_age * 100
matrix[2][2] = m/sum_of_age * 100
matrix[2][3] = n/sum_of_age * 100
matrix[2][4] = o/sum_of_age * 100
matrix[3][0] = p/sum_of_age * 100
matrix[3][1] = q/sum_of_age * 100
matrix[3][2] = r/sum_of_age * 100
matrix[3][3] = s/sum_of_age * 100
matrix[3][4] = t/sum_of_age * 100
matrix[4][0] = u/sum_of_age * 100
matrix[4][1] = v/sum_of_age * 100
matrix[4][2] = w/sum_of_age * 100
matrix[4][3] = x/sum_of_age * 100
matrix[4][4] = y/sum_of_age * 100
matrix


### END SOLUTION

### 3.11b Explore Suspect and Victim Sex Relationships
In this activity we explore relations between SUSP_SEX and VIC_SEX 
1. create a matrix of where rows are SUSP_SEX and Columns are VIC_SEX
2. Fill in the matrix with all probabilities (eg. "the probability that Males target other Males?")

In [ ]:
### BEGIN SOLUTION
import numpy as np
a = np.empty([2,2],dtype = float)
M = calls.groupby('SUSP_SEX')['VIC_SEX'].apply(pd.DataFrame)
mm = M['M'].value_counts()['M']
mf = M['M'].value_counts()['F']
ff = M['F'].value_counts()['F']
fm = M['F'].value_counts()['M']
sum_of_race = mm + mf + ff + fm
a[0][0] = mm/sum_of_race * 100
a[0][1] = mf/sum_of_race * 100
a[1][0] = fm/sum_of_race * 100
a[1][1] = ff/sum_of_race * 100
a

#MF = calls.groupby('SUSP_SEX').get_group('M').groupby('VIC_SEX').get_group('F')
#F = calls.groupby('SUSP_SEX').get_group('F').groupby('VIC_SEX').get_group('F')
#FM = calls.groupby('SUSP_SEX').get_group('F').groupby('VIC_SEX').get_group('M')
#M.value_counts()/F.value_counts()


### END SOLUTION

### 3.11c Observations 
Based on what was calculated in 3.11a and 3.11b, state at least important observations.
##### BEGIN SOLUTION
It seems that SUSP that whose ages are 25-44 have the biggest vitim rate, and the VIC they choose is also 25-44.SUPS who are too old or too young have a low probability. (Young people have their parents, and old people have their children or they are too old to move)
It seems that SUSP that whose race are male have a bigger probability, and the VIC they would like to choose most is female. The reason for this is that they think that females are more easily robbed I think.

##### END SOLUTION

### 3.12 - Visualization of crimes on a Map of NYC
Finally we attempt to visualize the crimes committed in NYC on a Map. For this activity, use all data, not just the First set,  NYPD_Complaint_Data_Historic_1. 
First we need to install some mapping software. RSun the cell below to install folium package for mapping software.

In [ ]:
!pip install --upgrade folium

In [ ]:
buNYC = calls[calls["OFNS_DESC"] == "BURGLARY"][:20]
len(buNYC)

In [ ]:
### Plot the maps for BURGLARY in NYC
### if it takes too much time or map does not show up, try plotting a subset of the data set of for a specific crime

### BEGIN SOLUTION
import folium
import json
import requests

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
map1 = folium.Map(location=[40,-73], zoom_start=7)


c = calls.dropna(subset=['Latitude', 'Longitude'])
calls = c[c['OFNS_DESC'] == 'BURGLARY'].reset_index()

for i in range(100):
    
    x = calls['Latitude'][i]
    y = calls['Longitude'][i]

    folium.Marker(
        location=[x,y],
        popup=folium.Popup(max_width=450).add_child(
            folium.Vega(json.loads(requests.get(f'{url}/vis1.json').text), width=450, height=250))
    ).add_to(map1)

### END SOLUTION

In [ ]:
map1

 ## Congratulations !!!

<div class="alert alert-block alert-info">
<h2>Submission Instructions</h2> 
<b> Output:</b> Please **remove all output** from your notebook prior to submission<br>
<b> File Name:</b> Please name the file as your_section_your_netID_Lab3.ipynb (eg. 01_adg133_Lab3.ipynb<br>
<b> Submit To: </b> Canvas &rarr; Assignments &rarr; Lab3 <br>
<b>Warning:</b> Failure to follow directions may result in loss of points.<br>
</div>

@2022 A.D. Gunawardena. Many people contributed to this lab including TA Liqin Long (now at Google). Much credits go to Josh Hug, and Berkeley Data Science Group for their contributions to the original version. Please DO NOT share this lab and/or post them on public sites.